<h1>วิธี Test
ให้วางโมเดลในไดร์ฟตัวเอง

<h1>สร้างโฟลเดอร์เพลงใส่เพลงmp3เข้าไป

<h1>แปลงเพลงเป็น.wav

<h1>แล้วก็ลองเลยได้ทีละเพลงนะ

<h1>เชื่อมไดร์ฟตัวเอง

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


<h1>แปลงเพลงที่จะให้testเป็น.wav

<h1>import

In [ ]:
!apt-get install ffmpeg -y
!pip install pydub
from pydub import AudioSegment
from pydub.utils import which

AudioSegment.converter = which("ffmpeg")  # บอกให้ pydub ใช้ ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
from pydub import AudioSegment
import os

# ฟังก์ชั่นสำหรับแปลงไฟล์ .mp3 เป็น .wav
def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio = audio.set_frame_rate(22050).set_channels(1)  # ตั้งค่า Sample Rate และ Mono
    audio.export(wav_path, format="wav")
    print(f"✅ แปลงไฟล์ {mp3_path} เป็น {wav_path} เรียบร้อย!")

# ตัวอย่างเส้นทางของไฟล์
mp3_folder = "/content/drive/My Drive/Project DE242 Music Genre Classification/Unseen_Data"  # โฟลเดอร์ที่เก็บไฟล์ .mp3
output_folder = "/content/drive/My Drive/Project DE242 Music Genre Classification/Unseen_Data_WAV"  # โฟลเดอร์ที่จะเก็บไฟล์ .wav

# สร้างโฟลเดอร์สำหรับไฟล์ .wav ถ้ายังไม่มี
os.makedirs(output_folder, exist_ok=True)

# แปลงไฟล์ทั้งหมดในโฟลเดอร์ .mp3 เป็น .wav
for file in os.listdir(mp3_folder):
    if file.endswith(".mp3"):
        mp3_path = os.path.join(mp3_folder, file)
        wav_path = os.path.join(output_folder, file.replace(".mp3", ".wav"))
        convert_mp3_to_wav(mp3_path, wav_path)

print("✅ การแปลงไฟล์เสร็จสิ้น!")


✅ แปลงไฟล์ /content/drive/My Drive/Project DE242 Music Genre Classification/Unseen_Data/ก่อนวันสุดท้าย · วุฒิ ป่าบอน พาราฮัท.mp3 เป็น /content/drive/My Drive/Project DE242 Music Genre Classification/Unseen_Data_WAV/ก่อนวันสุดท้าย · วุฒิ ป่าบอน พาราฮัท.wav เรียบร้อย!
✅ การแปลงไฟล์เสร็จสิ้น!


<h1>การทดสอบกับข้อมูลที่ไม่เคยเห็นมาก่อน (Unseen Data)

In [ ]:
import librosa
import numpy as np
import joblib  # ใช้ joblib สำหรับโหลดโมเดลที่ฝึกไว้
import os

# โหลดโมเดลที่เทรนไว้ (Random Forest)
model = joblib.load("/content/drive/My Drive/Project DE242 Music Genre Classification/random_forest_model.pkl")

# กำหนด label ของแนวเพลง
genres = ["Northern song", "Northeastern song", "Southern song", "Central song"]

# ฟังก์ชันดึงฟีเจอร์เสียง
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=22050)

    # ดึง MFCC, Chroma, Spectral Contrast
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).mean(axis=1)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr).mean(axis=1)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr).mean(axis=1)

    # รวมฟีเจอร์ทั้งหมด
    features = np.hstack([mfcc, chroma, spectral_contrast])
    return features

# 📂 โฟลเดอร์เก็บเพลงที่ใช้ทดสอบ (Unseen Data)
test_folder = "/content/drive/My Drive/Project DE242 Music Genre Classification/Unseen_Data_WAV"

# วนลูปผ่านไฟล์เสียงในโฟลเดอร์
for file in os.listdir(test_folder):
    if file.endswith(".wav"):
        file_path = os.path.join(test_folder, file)

        # ดึงฟีเจอร์
        features = extract_features(file_path)
        features = features.reshape(1, -1)  # Reshape ให้เข้ากับโมเดล

# ทำการพยากรณ์แนวเพลง
prediction = model.predict(features)

# Access the predicted label directly
predicted_label = prediction[0]

print(f"🎵 ไฟล์: {file} → ทำนายว่าเป็นแนวเพลง: {predicted_label}")


🎵 ไฟล์: ก่อนวันสุดท้าย · วุฒิ ป่าบอน พาราฮัท.wav → ทำนายว่าเป็นแนวเพลง: Northeastern song


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


<h1>เว็ปHTML

In [ ]:
# ติดตั้งไลบรารีที่จำเป็น
!pip install flask flask-cors pyngrok pydub librosa scikit-learn

# แล้วใช้ pyngrok แบบนี้แทน flask_ngrok
from pyngrok import ngrok


In [ ]:
# เพิ่มคำสั่งติดตั้ง ngrok ก่อนเรียกใช้ Flask
!pip install pyngrok
!ngrok authtoken YOUR_AUTH_TOKEN  # รับ token จาก https://dashboard.ngrok.com/get-started/your-authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import numpy as np
import librosa
import joblib
from pydub import AudioSegment
from pydub.utils import which

# ตั้งค่า Ngrok
ngrok.set_auth_token("2vDIG7SNYtNCgxajjckTsNcV2td_3m79MR3Hp7neCpkK7tTKZ")  # ใส่ Auth Token จริงของคุณ
public_url = ngrok.connect(5000).public_url
AudioSegment.converter = which("ffmpeg")

app = Flask(__name__)
CORS(app)

# โหลดโมเดลตรงนี้
model_path = "/content/drive/My Drive/Project DE242 Music Genre Classification/random_forest_model.pkl"
model = joblib.load(model_path)
print(f"✅ โหลดโมเดลสำเร็จจาก: {model_path}")

# รายชื่อแนวเพลง
genres = ["Northern song", "Northeastern song", "Southern song", "Central song"]

# ฟังก์ชันดึงฟีเจอร์เสียง
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).mean(axis=1)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr).mean(axis=1)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr).mean(axis=1)
    return np.hstack([mfcc, chroma, spectral_contrast])

@app.route("/predict", methods=["POST"])
def predict():
    try:
        file = request.files["file"]
        temp_path = f"/tmp/{file.filename}"
        file.save(temp_path)

        # แปลงเป็น .wav ถ้าเป็น mp3
        if file.filename.endswith(".mp3"):
            wav_path = temp_path.replace(".mp3", ".wav")
            audio = AudioSegment.from_mp3(temp_path)
            audio = audio.set_frame_rate(22050).set_channels(1)
            audio.export(wav_path, format="wav")
            os.remove(temp_path)
            temp_path = wav_path

        features = extract_features(temp_path)
        features = features.reshape(1, -1)

        prediction = model.predict(features)
        predicted_genre = prediction[0]


        os.remove(temp_path)

        return jsonify({'genre': predicted_genre, 'status': 'success'})

    except Exception as e:
        print("🔥 Error:", str(e))
        return jsonify({'error': str(e)}), 500

# pyngrok tunnel
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("🌐 Public URL:", public_url)

app.run()


✅ โหลดโมเดลสำเร็จจาก: /content/drive/My Drive/Project DE242 Music Genre Classification/random_forest_model.pkl
🌐 Public URL: NgrokTunnel: "https://4416-35-197-72-5.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [22/Apr/2025 05:20:56] "POST /predict HTTP/1.1" 200 -
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [22/Apr/2025 06:28:31] "POST /predict HTTP/1.1" 200 -
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [22/Apr/20